In [1]:
##############################
#### Forest plot
##############################
library(data.table) # version 1.14.6
library(meta) # version 6.2-1
library(grid) # version 4.2.2
library(scales) # version 1.2.1
# set working directory
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/epi/")


Loading 'meta' package (version 4.18-2).
Type 'help(meta)' for a brief overview.



In [2]:
## Forest Plot
glm_smoking_bmi <- fread("final_glm.multivariable_smoking_bmi_cat.incident_ch.2023Jul07.csv", header = T)
table(glm_smoking_bmi$Exposure)
glm_smoking_bmi$Exposure[glm_smoking_bmi$Exposure=="Former_smoker"] <- "Former smoker"
glm_smoking_bmi$Exposure[glm_smoking_bmi$Exposure=="Current_smoker"] <- "Current smoker"
glm_smoking_bmi$Exposure[glm_smoking_bmi$Exposure=="BMI_25-30"] <- "BMI: 25-30"
glm_smoking_bmi$Exposure[glm_smoking_bmi$Exposure=="BMI_>30"] <- "BMI: >30"
table(glm_smoking_bmi$Exposure)

glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_CH"] <- "Overall CH"
glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_DNMT3A"] <- "DNMT3A"
glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_TET2"] <- "TET2"
glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_ASXL1"] <- "ASXL1"
glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_SF"] <- "SF"
glm_smoking_bmi$Outcome[glm_smoking_bmi$Outcome=="incident_DDR"] <- "DDR"

table(glm_smoking_bmi$Outcome)


     BMI_25-30        BMI_>30 Current_smoker  Former_smoker 
             6              6              6              6 


    BMI: 25-30       BMI: >30 Current smoker  Former smoker 
             6              6              6              6 


     ASXL1        DDR     DNMT3A Overall CH         SF       TET2 
         4          4          4          4          4          4 

In [3]:
glm_smoking_bmi$Outcome <- ordered(glm_smoking_bmi$Outcome, 
                           levels = c("Overall CH", 
                                      "DNMT3A", "TET2", 
                                      "ASXL1", 
                                      "SF", "DDR")) 

## exposure
glm_smoking_bmi$Exposure <- ordered(glm_smoking_bmi$Exposure, 
                            levels = c("Former smoker", "Current smoker", 
                                       "BMI: 25-30", "BMI: >30"))

In [4]:
## Format
glm_smoking_bmi$P_val <- formatC(x = glm_smoking_bmi$P, digits = 1,format = "E")

# OR
glm_smoking_bmi$OR <- formatC(round(exp(glm_smoking_bmi$Beta),2), digits = 2, format = "f")

glm_smoking_bmi$lSE <- ( glm_smoking_bmi$Beta - 1.96 * glm_smoking_bmi$SE)
glm_smoking_bmi$uSE <- ( glm_smoking_bmi$Beta + 1.96 * glm_smoking_bmi$SE)

# 95% CI
glm_smoking_bmi$CI95 <- paste0("[",formatC(round(exp( glm_smoking_bmi$Beta - 1.96 * glm_smoking_bmi$SE),2), digits = 2, format = "f"),
                       ", ",
                       formatC(round(exp( glm_smoking_bmi$Beta + 1.96 * glm_smoking_bmi$SE),2), digits = 2, format = "f"), 
                       "]")
head(glm_smoking_bmi)

glm_smoking_bmi$sig <-  ifelse(glm_smoking_bmi$P<0.0025, "***","")


Dataset,Outcome,Exposure,Beta,SE,t-stat,P,P_val,OR,lSE,uSE,CI95
<chr>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Adjusted,Overall CH,Former smoker,0.01627726,0.0953649,0.17068393,0.8644723,8.6E-01,1.02,-0.17063794,0.2031925,"[0.84, 1.23]"
Adjusted,Overall CH,Current smoker,0.11964492,0.1274053,0.93908908,0.3476850,3.5E-01,1.13,-0.13006946,0.3693593,"[0.88, 1.45]"
Adjusted,Overall CH,BMI: 25-30,0.16065404,0.1043260,1.53992384,0.1235789,1.2E-01,1.17,-0.04382485,0.3651329,"[0.96, 1.44]"
Adjusted,Overall CH,BMI: >30,-0.06287942,0.1254613,-0.50118570,0.6162404,6.2E-01,0.94,-0.30878359,0.1830248,"[0.73, 1.20]"
Adjusted,DNMT3A,Former smoker,0.00850073,0.1359200,0.06254217,0.9501311,9.5E-01,1.01,-0.25790241,0.2749039,"[0.77, 1.32]"
Adjusted,DNMT3A,Current smoker,-0.12175997,0.1880153,-0.64760679,0.5172393,5.2E-01,0.89,-0.49026993,0.2467500,"[0.61, 1.28]"


In [5]:
##### Adjusted model: never vs Former/ current smoker 
# BMI <25 vs 25-30 / >30
b_smk_bmi <- metagen(TE = Beta,
                    lower = lSE,
                    upper = uSE,
                    studlab = Outcome,
                    subgroup=Exposure,
                    data=glm_smoking_bmi,
                    sm="OR")


pdf("figS5.final_glm.smoking_bmi_cat.Forest_incidentCH.2023Jul07.pdf",
    width = 8, height= 8)
forest(x = b_smk_bmi, 
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="same",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of Exposures\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio", 
       squaresize=0.6, 
       col.subgroup="black", 
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val", "sig"),
       rightlabs=c("OR","95% CI","P", ""),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()

ERROR: Error in metagen(TE = Beta, lower = lSE, upper = uSE, studlab = Outcome, : unused argument (subgroup = Exposure)
